In [1]:
from collections import Counter
import pandas as pd
import numpy as np
import re

# *1. Jaccad 유사도*  
![Jaccad 유사도](./images/jaccad_sim.png)  

# *2. Cosine 유사도*
![Cosine 유사도](./images/cosine_sim.png)  

In [2]:
## 단어 전처리해주는 함수 | 정규 표현식으로 알파벳과 숫자만 남기도록 함
word_preprocessing = lambda word: re.sub('[^a-zA-Z0-9]', '', word)

class similarity:

    def __init__(self, word1, word2):
        self.word1 = word1
        self.word2 = word2
    
    
    ## 단어 전처리해주는 함수 | 정규 표현식으로 알파벳과 숫자, 한글만 남기도록 함
    def preprocessing(self, word): 
        return re.sub('[^a-zA-Z0-9가-힣]', '', word)

    
    ## 단어를 벡터화 시키는 함수
    def word_vectorizer(self, word):
        word = self.preprocessing(word)
        word_count = Counter(word.lower())

        ##     ascii | 0 ~ 9, a ~ z, 한국어 까지의 문자열을 리스트에 저장 (ascii 코드표에 대응시킴)
        ascii = [chr(idx) for idx in range(48, 58)] + [chr(idx) for idx in range(97, 127)] + [chr(idx) for idx in range(44032, 55204)]
        alphabets = {alpha: 0 for alpha in ascii}

        ## 단어에 있는 알파벳 갯수만을 반환시켜줌.
        alphabets = {**alphabets, **word_count}
        return np.array([num for num in alphabets.values()])

    def tf_idf(self, vectors, columns):
        print(f'tf : {vectors[0]} {len(vectors[0])}\n\n')
        
        tf = np.array([np.divide(np.array([num for num in vector.values()]), len(vector)) for vector in vectors])
        print(f'tf : {tf}\n\n')
        df = {col : 0 for col in columns}
        
        for vector in vectors:
            print(columns, vector.keys())
            for col, word in zip(columns, vector.keys()):
                if col in word: df[col] += 1
                else: vector[col] = 0
                
        
        df = np.array([val for val in df.values()])
        df = np.divide(df, len(vectors))        
        
        idf = np.log(len(vectors) / (1 + df))    
        return np.array([np.multiply(vector, idf) for vector in vectors])
    
    
    ## 텍스트를 벡터화 시키는 함수
    def text_vectorizer(self, method='bow'):
        
        ## 공백을 기준으로 텍스트를 나눔.
        word1, word2 = self.word1.split(' '), self.word2.split(' ')
        
        ## 정규 표현식으로 알파벳과 숫자, 한글만 남기도록 전처리.
        word1 = [self.preprocessing(word) for word in word1]
        word2 = [self.preprocessing(word) for word in word2]
        
        ## 정규표현식을 통해 삭제되어 생성된 공백문자 제거
        ## e.g)                   words = ['abcd', '!@#$%', 'defg']
        ##      (정규표현식 거침) words = ['abcd', '', 'defg']
        ##           (공백 제거) words = ['abcd', 'defg']
        
        words1 = [word for word in word1 if word not in '']
        words2 = [word for word in word2 if word not in '']

        ## 첫 번째 텍스트와 두 번째 텍스트에 있는 단어들로 딕셔너리 생성
        words1_dict, words2_dict = {word: 0 for word in words1}, {word: 0 for word in words2}
        
        ## {텍스트 : 개수}로 구성된 딕셔너리
        words1, words2 = Counter(words1), Counter(words2)
        
        ## words1_dict와 words2_dict를 합쳐 하나의 딕셔너리로 생성
        vector1, vector2 = {**words1_dict, **words2_dict},  {**words1_dict, **words2_dict}
        
        ## vector 딕셔너리에 텍스트에 있는 단어 개수 입력하는 부분
        for word, num in words1.items(): vector1[word] = num
        for word, num in words2.items(): vector2[word] = num

        column = vector1.keys()
        
        ## 텍스트 벡터화 시켜주는 부분
        vector1 = np.array([num for num in vector1.values()])
        vector2 = np.array([num for num in vector2.values()])
        
        ## TF(Term Frequency) | 문서 d에서 단어 t의 출현 빈도
        BoW = pd.DataFrame([vector1, vector2], index = ['text1', 'text2'], columns = column)
        
        if 'bow' in method.lower(): return np.array([vector1, vector2]), BoW
        elif 'tf-idf' in method.lower(): return np.array([words1, words2]), BoW

    
    ## Jaccad 유사도 구하는 함수
    def jaccad_sim(self):

        word1 = self.preprocessing(self.word1)
        word2 = self.preprocessing(self.word2)

        intersection = set(word1).intersection(set(word2))
        union        = set(word1).union(set(word2))

        return len(intersection) / len(union)


    ## 코사인 유사도 구하는 함수
    def cosine_sim(self, task = 'word', method = 'BoW'):

        if 'word' in task.lower():
          ## 입력받은 단어 두 개를 벡터화 시켜줌.
          vector1 = self.word_vectorizer(self.word1)
          vector2 = self.word_vectorizer(self.word2)
            
        elif 'text' in task.lower():
            vectors, TF = self.text_vectorizer(method = method)
            
            if 'bow' in method.lower():
                vector1, vector2 = vectors[0], vectors[1]
                
            elif 'tf-idf' in method.lower():
                tf_idf = self.tf_idf(vectors, TF.columns.to_list())
                vector1, vector2 = tf_idf[0], tf_idf[1]
                
            print(f'\n\n tf_idf1 : \n{vector1} \n\n tf_idf2 :\n{vector2}\n')
        
        ## 코사인 유사도 구해주는 부분
        try: cosine = np.dot(vector1, vector2) / (np.linalg.norm(vector1)*np.linalg.norm(vector2))
        except: cosine = 0.0

        return cosine

In [3]:
sim = similarity('calcif', 'calcification')
sim2 = similarity('calclifcation', 'calcification')

In [4]:
jaccad1 = sim.jaccad_sim()
jaccad2 = sim2.jaccad_sim()

cosine1 = sim.cosine_sim()
cosine2 = sim2.cosine_sim()

print(f'jaccad1 : {jaccad1}, cosine1 : {cosine1}')
print(f'jaccad2 : {jaccad2}, cosine2 : {cosine2}')

jaccad1 : 0.625, cosine1 : 0.884537962671703
jaccad2 : 1.0, cosine2 : 0.9622504486493763


In [5]:
# %%time
sim3 = similarity('if you take the blue pill, the story ends', 'if you take the red pill, you stay in Wonderland')

(vec1, vec2), BoW = sim3.text_vectorizer()
print(sim3.cosine_sim(task = 'text'))
BoW



 tf_idf1 : 
[1 1 1 2 1 1 1 1 0 0 0 0] 

 tf_idf2 :
[1 2 1 1 0 1 0 0 1 1 1 1]

0.6092717958449424


,if,you,take,the,blue,pill,story,ends,red,stay,in,Wonderland
text1,1,1,1,2,1,1,1,1,0,0,0,0
text2,1,2,1,1,0,1,0,0,1,1,1,1


In [6]:
sim4 = similarity('calcification', 'calcifi')
sim4.cosine_sim(task='word')

0.9284141650970551

In [12]:
sim5 = similarity('김둘기는 쌀알을 쪼아 먹습니다. 그리고 말했습니다. "나 이거 정말 맛시써!"', '김참새는 땅콩을 쪼아 먹습니다. 그리고 말했습니다. "나 이거 정말 쪼아!"')

(vec1, vec2), BoW = sim5.text_vectorizer()
print(sim5.cosine_sim(task='text'))
BoW



 tf_idf1 : 
[1 1 1 1 1 1 1 1 1 1 0 0] 

 tf_idf2 :
[0 0 2 1 1 1 1 1 1 0 1 1]

0.7302967433402214


,김둘기는,쌀알을,쪼아,먹습니다,그리고,말했습니다,나,이거,정말,맛시써,김참새는,땅콩을
text1,1,1,1,1,1,1,1,1,1,1,0,0
text2,0,0,2,1,1,1,1,1,1,0,1,1
